# Exercise 1

## Imports

In [ ]:
import os.path
import pyspark.sql.functions as F
from pyspark import Broadcast
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, ArrayType, FloatType
from itertools import combinations
from typing import Iterable, Any, List

import pandas as pd
import numpy as np
import math

## Spark initialization

In [ ]:
spark = SparkSession.builder \
    .appName('exercise1') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

## Prepare the data

In [ ]:
tracks_df = (spark.read
    .option("multiline", "true")
    .option("quote", '"')
    .option("escape", '"')
    .csv('data/tracks.csv')
)

# rename columns with row values from first row to second row
column_categories = zip(*tracks_df.take(2))
columns = tracks_df.columns
tracks_df = tracks_df.select(columns[0].alias('track_id'),
    *(column.cast(FloatType()).alias("-".join(map(str, categories)))
    for column, categories in zip(columns[1:], column_categories[1:]))
)

tracks_df = (tracks_df
    .filter(F.col("track_id").isNotNull()) 
    .filter(F.col("track_id") != "track_id")
)

tracks_df.show()

In [ ]:
features_df = (spark.read
    .csv('data/features.csv')
)

# rename columns with row values from first row to second row
column_categories = zip(*features_df.take(3))
columns = features_df.columns
features_df = features_df.select(columns[0].alias('track_id'),
    *(column.cast(FloatType()).alias("-".join(map(str, categories)))
    for column, categories in zip(columns[1:], column_categories[1:]))
)

features_df = (features_df
    .filter(F.col("track_id") != "feature")
    .filter(F.col("track_id") != "statistics")
    .filter(F.col("track_id") != "number")
    .filter(F.col("track_id") != "track_id")
)

## Agglomerative clustering (in-memory)

In [ ]:
small_tracks_df = tracks_df.filter(F.col("set-subset") == "small")
small_features_df = (features_df
    .join(small_tracks_df, "track_id", "left")
    .filter(F.col("set-subset").isNotNull())
    .select(features_df.columns)
)

music_features_pd = (small_features_df
    .drop("track_id")
    .toPandas()
)
music_features_pd = music_features_pd.astype(np.float64)

In [ ]:
# calculate the metrics (radius, diameter, density_r, density_d) for each cluster
def calculate_metrics(pd_df,centroids):
    
    cluster = pd_df["cluster"].values[0]
    
    metrics = pd.DataFrame({'radius': [0], 'diameter': [0],'density_r': [0],'density_d': [0]},columns=['radius', 'diameter','density_r','density_d'])

    centroid = centroids[cluster]

    for index, row in pd_df.iterrows():

        #row to array 
        a = row.to_numpy()

        radius = math.dist(a,centroid)

        if radius > metrics.loc[0,'radius']:
            metrics.loc[0,'radius'] = radius

    # calculater density with radius
    metrics.loc[0,'density_r'] = pd_df.size / metrics.loc[0,'radius']**2

    all_combinations = list(combinations(range(0,pd_df.size), 2))

    for combination in all_combinations:

        a = pd_df.iloc[combination[0]].to_numpy()
        b = pd_df.iloc[combination[1]].to_numpy()

        diameter = math.dist(a,b)

        if diameter > metrics.loc[0,'diameter']:
            metrics.loc[0,'diameter'] = diameter
        
    # calculater density with diameter
    metrics.loc[0,'density_d'] = pd_df.size / metrics.loc[0,'diameter']**2

    return metrics
    

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid

metrics_pd_array = []

# i = 8 until 16
for i in range(8, 17):
    n_clusters = i
    clusterer = AgglomerativeClustering(n_clusters=n_clusters, compute_distances=True)
    clusterer.fit(music_features_pd)
    print(f"n_clusters: {n_clusters}, cluster_labels: {clusterer.labels_}")

    # calculate centroids

    centroid_calculator = NearestCentroid()

    centroid_calculator.fit(music_features_pd, clusterer.labels_)

    column_categories = music_features_pd
    column_categories["cluster"] = clusterer.labels_
    
    metrics_pd_array.append(column_categories.groupby("cluster").apply(calculate_metrics,centroid_calculator.centroids_))